# SDA - lecture 5 - Multiple point processes

In [ ]:
import logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(asctime)s: %(message)s')

import numpy as np
import matplotlib.pyplot as plt
import pycorrelate

%matplotlib inline

### Generate two spike trains

In [ ]:
# Generate a "spike train" of a neuron 
samp = 1000
duration = 600
rate_A = 2
rate_B = 5

# Generate a regular neuron A
interval = samp/rate_A
spk_time_A = np.arange(interval/2,duration*samp, interval).astype(np.int32)
spk_array_A = np.zeros(duration*samp)
spk_array_A[spk_time_A] = 1

# Generate an affected neuron B
alpha = 0.04
t = np.arange(-300,301)
alpha_array = (rate_B/rate_A)*alpha**2*(t*(t>0))*np.exp(-alpha*t)
rate_array_B = np.convolve(spk_array_A, alpha_array, 'same')
spk_array_B = (np.random.uniform(size=samp*duration)<rate_array_B).astype(np.int32)
time_array = np.arange(0, duration, 1/samp)

In [ ]:
plot_duration = 10000
fig, ax = plt.subplots(figsize=(12,4), nrows=2, ncols=1)
ax[0].plot(time_array[0:plot_duration], spk_array_A[0:plot_duration])
ax[0].set_ylabel('Spikes')
ax[0].set_title('Neuron A (regular)')
ax[1].plot(time_array[0:plot_duration], spk_array_B[0:plot_duration])
ax[1].set_ylabel('Spikes')
ax[1].set_xlabel('Time [s]')
ax[1].set_title('Neuron B')
logging.info(f'Rate of neuron A: {spk_array_A.sum()/duration:.1f} spikes/s')
logging.info(f'Rate of neuron B: {spk_array_B.sum()/duration:.1f} spikes/s')

### Draw the cross correlation

In [ ]:
def plt_corr(c, ax=ax):
    ax.plot(cbins[:-1],c / duration * samp)
    ax.set_title('Cross correlation (A-->B)')
    ax.set_ylabel('Rate [spikes/s]')
    ax.set_xlabel('Offset [ms]')

fig, ax = plt.subplots(figsize=(12,3), nrows=1, ncols=1)
cbins = np.arange(-1000,1000,1)
xcorr = pycorrelate.pcorrelate(spk_array_A.nonzero()[0], spk_array_B.nonzero()[0], bins=cbins)
plt_corr(xcorr, ax)

In [ ]:
fig, ax = plt.subplots(figsize=(12,3), nrows=1, ncols=1)
cbins = np.arange(-250,250,1)
xcorr = pycorrelate.pcorrelate(spk_array_A.nonzero()[0], spk_array_B.nonzero()[0], bins=cbins)
plt_corr(xcorr, ax)